In [42]:
from enum import Enum
import os
import pickle
from typing import List
import pandas as pd

from openai import OpenAI
from pydantic import BaseModel

In [2]:
class categories(str, Enum):
    OTHER = "Other"

    # Amplifying Climate Fears
    CC_AMPLIFY_FEAR_EXISTING = "CC: Amplifying Climate Fears: Amplifying existing fears of global warming"
    CC_AMPLIFY_FEAR_DOOMSDAY = "CC: Amplifying Climate Fears: Doomsday scenarios for humans"
    CC_AMPLIFY_FEAR_UNINHABITABLE = "CC: Amplifying Climate Fears: Earth will be uninhabitable soon"
    CC_AMPLIFY_FEAR_OTHER = "CC: Amplifying Climate Fears: Other"
    CC_AMPLIFY_FEAR_TOO_LATE = "CC: Amplifying Climate Fears: Whatever we do it is already too late"
    
    # Climate Change is Beneficial
    CC_BENEFICIAL_CO2 = "CC: Climate change is beneficial: CO2 is beneficial"

    # Controversy About Green Technologies
    CC_GREEN_TECH_COSTLY = "CC: Controversy about green technologies: Renewable energy is costly"
    CC_GREEN_TECH_DANGEROUS = "CC: Controversy about green technologies: Renewable energy is dangerous"
    CC_GREEN_TECH_UNRELIABLE = "CC: Controversy about green technologies: Renewable energy is unreliable"
    CC_GREEN_TECH_OTHER = "CC: Controversy about green technologies: Other"

    # Criticism of Climate Movement
    CC_CRITICISM_MOVEMENT_ACTIVISTS = "CC: Criticism of climate movement: Ad hominem attacks on key activists"
    CC_CRITICISM_MOVEMENT_ALARMIST = "CC: Criticism of climate movement: Climate movement is alarmist"
    CC_CRITICISM_MOVEMENT_CORRUPT = "CC: Criticism of climate movement: Climate movement is corrupt"
    CC_CRITICISM_MOVEMENT_OTHER = "CC: Criticism of climate movement: Other"

    # Criticism of Climate Policies
    CC_CRITICISM_POLICIES_INEFFECTIVE = "CC: Criticism of climate policies: Climate policies are ineffective"
    CC_CRITICISM_POLICIES_PROFIT = "CC: Criticism of climate policies: Climate policies are only for profit"
    CC_CRITICISM_POLICIES_ECONOMIC_IMPACT = "CC: Criticism of climate policies: Climate policies have negative impact on the economy"
    CC_CRITICISM_POLICIES_OTHER = "CC: Criticism of climate policies: Other"

    # Criticism of Institutions and Authorities
    CC_CRITICISM_AUTHORITIES_INTERNATIONAL = "CC: Criticism of institutions and authorities: Criticism of international entities"
    CC_CRITICISM_AUTHORITIES_GOVERNMENTS = "CC: Criticism of institutions and authorities: Criticism of national governments"
    CC_CRITICISM_AUTHORITIES_POLITICAL = "CC: Criticism of institutions and authorities: Criticism of political organizations and figures"
    CC_CRITICISM_AUTHORITIES_EU = "CC: Criticism of institutions and authorities: Criticism of the EU"
    CC_CRITICISM_AUTHORITIES_OTHER = "CC: Criticism of institutions and authorities: Other"

    # Downplaying Climate Change
    CC_DOWNPLAY_CO2_CONCENTRATIONS = "CC: Downplaying climate change: CO2 concentrations are too small to have an impact"
    CC_DOWNPLAY_NATURAL_CYCLES = "CC: Downplaying climate change: Climate cycles are natural"
    CC_DOWNPLAY_HUMAN_IMPACT = "CC: Downplaying climate change: Human activities do not impact climate change"
    CC_DOWNPLAY_ADAPTATION = "CC: Downplaying climate change: Humans and nature will adapt to the changes"
    CC_DOWNPLAY_ICE_NOT_MELTING = "CC: Downplaying climate change: Ice is not melting"
    CC_DOWNPLAY_OTHER = "CC: Downplaying climate change: Other"
    CC_DOWNPLAY_TEMP_IMPACT = "CC: Downplaying climate change: Temperature increase does not have significant impact"
    CC_DOWNPLAY_COOLING_TREND = "CC: Downplaying climate change: Weather suggests the trend is global cooling"

    # Green Policies as Geopolitical Instruments
    CC_GEO_POLICIES_NEO_COLONIALISM = "CC: Green policies are geopolitical instruments: Green activities are a form of neo-colonialism"
    CC_GEO_POLICIES_OTHER = "CC: Green policies are geopolitical instruments: Other"

    # Hidden Plots by Secret Schemes of Powerful Groups
    CC_PLOTS_GLOBAL_ELITES = "CC: Hidden plots by secret schemes of powerful groups: Blaming global elites"
    CC_PLOTS_HIDDEN_MOTIVES = "CC: Hidden plots by secret schemes of powerful groups: Climate agenda has hidden motives"
    CC_PLOTS_OTHER = "CC: Hidden plots by secret schemes of powerful groups: Other"

    # Questioning the Measurements and Science
    CC_QUESTION_DATA_NO_INCREASE = "CC: Questioning the measurements and science: Data shows no temperature increase"
    CC_QUESTION_GREENHOUSE_EFFECT = "CC: Questioning the measurements and science: Greenhouse effect/carbon dioxide do not drive climate change"
    CC_QUESTION_METHODS_UNRELIABLE = "CC: Questioning the measurements and science: Methodologies/metrics used are unreliable/faulty"
    CC_QUESTION_OTHER = "CC: Questioning the measurements and science: Other"
    CC_QUESTION_COMMUNITY_UNRELIABLE = "CC: Questioning the measurements and science: Scientific community is unreliable"

    # Amplifying War-Related Fears
    WWIII = "URW: Amplifying war-related fears: By continuing the war we risk WWIII"
    NATO_INTERVENE = "URW: Amplifying war-related fears: NATO should/will directly intervene"
    WAR_FEARS_OTHER = "URW: Amplifying war-related fears: Other"
    RUSSIA_ATTACK_COUNTRIES = "URW: Amplifying war-related fears: Russia will also attack other countries"
    NUKES = "URW: Amplifying war-related fears: There is a real possibility that nuclear weapons will be employed"
    
    # Blaming the War on Others Rather than the Invader
    NOT_INVADER_OTHER = "URW: Blaming the war on others rather than the invader: Other"
    WEST_AGGRESSORS = "URW: Blaming the war on others rather than the invader: The West are the aggressors"
    UKRAINE_AGGRESSORS = "URW: Blaming the war on others rather than the invader: Ukraine is the aggressor"
    
    # Discrediting Ukraine
    DISCREDIT_UA_GOVERNMENT = "URW: Discrediting Ukraine: Discrediting Ukrainian government and officials and policies"
    DISCREDIT_UA_MILITARY = "URW: Discrediting Ukraine: Discrediting Ukrainian military"
    DISCREDIT_UA_SOCIETY = "URW: Discrediting Ukraine: Discrediting Ukrainian nation and society"
    DISCREDIT_UA_OTHER = "URW: Discrediting Ukraine: Other"
    DISCREDIT_UA_HISTORY = "URW: Discrediting Ukraine: Rewriting Ukraine’s history"
    DISCREDIT_UA_HOPELESS = "URW: Discrediting Ukraine: Situation in Ukraine is hopeless"
    DISCREDIT_UA_CRIMINAL = "URW: Discrediting Ukraine: Ukraine is a hub for criminal activities"
    DISCREDIT_UA_PUPPET = "URW: Discrediting Ukraine: Ukraine is a puppet of the West"
    DISCREDIT_UA_NAZISM = "URW: Discrediting Ukraine: Ukraine is associated with nazism"
    
    # Discrediting the West, Diplomacy
    DISCREDIT_WEST_DIPLOMACY_FAIL = "URW: Discrediting the West, Diplomacy: Diplomacy does/will not work"
    DISCREDIT_WEST_DIPLOMACY_OTHER = "URW: Discrediting the West, Diplomacy: Other"
    DISCREDIT_WEST_DIVIDED = "URW: Discrediting the West, Diplomacy: The EU is divided"
    DISCREDIT_WEST_INTERESTS = "URW: Discrediting the West, Diplomacy: The West does not care about Ukraine, only about its interests"
    DISCREDIT_WEST_OVERREACTING = "URW: Discrediting the West, Diplomacy: The West is overreacting"
    DISCREDIT_WEST_WEAK = "URW: Discrediting the West, Diplomacy: The West is weak"
    DISCREDIT_WEST_TIRED = "URW: Discrediting the West, Diplomacy: West is tired of Ukraine"
    
    # Distrust Towards Media
    DISTRUST_MEDIA_OTHER = "URW: Distrust towards Media: Other"
    DISTRUST_MEDIA_UA = "URW: Distrust towards Media: Ukrainian media cannot be trusted"
    DISTRUST_MEDIA_WEST = "URW: Distrust towards Media: Western media is an instrument of propaganda"
    
    # Hidden Plots by Secret Schemes of Powerful Groups
    PLOTS_OTHER = "URW: Hidden plots by secret schemes of powerful groups: Other"
    
    # Negative Consequences for the West
    NEGATIVE_WEST_OTHER = "URW: Negative Consequences for the West: Other"
    NEGATIVE_WEST_SANCTIONS_BACKFIRE = "URW: Negative Consequences for the West: Sanctions imposed by Western countries will backfire"
    NEGATIVE_WEST_REFUGEES = "URW: Negative Consequences for the West: The conflict will increase the Ukrainian refugee flows to Europe"
    
    # Overpraising the West
    OVERPRAISING_WEST_OTHER = "URW: Overpraising the West: Other"
    OVERPRAISING_WEST_HISTORY = "URW: Overpraising the West: The West belongs in the right side of history"
    OVERPRAISING_WEST_SUPPORT = "URW: Overpraising the West: The West has the strongest international support"
    
    # Praise of Russia
    PRAISE_RUSSIA_OTHER = "URW: Praise of Russia: Other"
    PRAISE_RUSSIA_PUTIN = "URW: Praise of Russia: Praise of Russian President Vladimir Putin"
    PRAISE_RUSSIA_MILITARY = "URW: Praise of Russia: Praise of Russian military might"
    PRAISE_RUSSIA_SUPPORT = "URW: Praise of Russia: Russia has international support from a number of countries and people"
    PRAISE_RUSSIA_PEACE = "URW: Praise of Russia: Russia is a guarantor of peace and prosperity"
    PRAISE_RUSSIA_INVASION_SUPPORT = "URW: Praise of Russia: Russian invasion has strong national support"
    
    # Russia is the Victim
    RUSSIA_VICTIM_OTHER = "URW: Russia is the Victim: Other"
    RUSSIA_SELF_DEFENSE = "URW: Russia is the Victim: Russia actions in Ukraine are only self-defence"
    RUSSIA_RUSSO_PHOBIC = "URW: Russia is the Victim: The West is russophobic"
    RUSSIA_ANTI_RU = "URW: Russia is the Victim: UA is anti-RU extremists"
    
    # Speculating War Outcomes
    WAR_OUTCOMES_OTHER = "URW: Speculating war outcomes: Other"
    WAR_OUTCOMES_RU_COLLAPSING = "URW: Speculating war outcomes: Russian army is collapsing"
    WAR_OUTCOMES_UA_COLLAPSING = "URW: Speculating war outcomes: Ukrainian army is collapsing"

In [41]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

class GENERALMODEL(BaseModel):
    narratives: List[categories]
    explanation: str

In [38]:
# Prompt:
# add this? A lot of the texts are classified as Other, so make sure the classes really fit the text.

classifier_prompt = """
You are an AI model, tasked with the responsibility of classifying narratives in news articles. 
The news articles are either about Climate Change, or the Ukraine Russia War. 
You must find all relevant sub-narratives from the domain that are applicable to the news article.
If none of the sub-narratives match, you must output "Other". If you choose Other, no other sub narratives can be chosen. 
Lastly, we would like you to provide a short explanation of max 80 words as to why you have chosen that particular narrative.

The URW (Ukraine Russia War) sub-narratives are the following:
'Other'
'URW: Amplifying war-related fears: By continuing the war we risk WWIII'
'URW: Amplifying war-related fears: NATO should/will directly intervene'
'URW: Amplifying war-related fears: Other'
'URW: Amplifying war-related fears: Russia will also attack other countries'
'URW: Amplifying war-related fears: There is a real possibility that nuclear weapons will be employed'
'URW: Blaming the war on others rather than the invader: Other'
 'URW: Blaming the war on others rather than the invader: The West are the aggressors'
 'URW: Blaming the war on others rather than the invader: Ukraine is the aggressor'
 'URW: Discrediting Ukraine: Discrediting Ukrainian government and officials and policies'
 'URW: Discrediting Ukraine: Discrediting Ukrainian military'
 'URW: Discrediting Ukraine: Discrediting Ukrainian nation and society'
 'URW: Discrediting Ukraine: Other'
 'URW: Discrediting Ukraine: Rewriting Ukraine’s history'
 'URW: Discrediting Ukraine: Situation in Ukraine is hopeless'
 'URW: Discrediting Ukraine: Ukraine is a hub for criminal activities'
 'URW: Discrediting Ukraine: Ukraine is a puppet of the West'
 'URW: Discrediting Ukraine: Ukraine is associated with nazism'
 'URW: Discrediting the West, Diplomacy: Diplomacy does/will not work'
 'URW: Discrediting the West, Diplomacy: Other'
 'URW: Discrediting the West, Diplomacy: The EU is divided'
 'URW: Discrediting the West, Diplomacy: The West does not care about Ukraine, only about its interests'
 'URW: Discrediting the West, Diplomacy: The West is overreacting'
 'URW: Discrediting the West, Diplomacy: The West is weak'
 'URW: Discrediting the West, Diplomacy: West is tired of Ukraine'
 'URW: Distrust towards Media: Other'
 'URW: Distrust towards Media: Ukrainian media cannot be trusted'
 'URW: Distrust towards Media: Western media is an instrument of propaganda'
 'URW: Hidden plots by secret schemes of powerful groups: Other'
 'URW: Negative Consequences for the West: Other'
 'URW: Negative Consequences for the West: Sanctions imposed by Western countries will backfire'
 'URW: Negative Consequences for the West: The conflict will increase the Ukrainian refugee flows to Europe'
 'URW: Overpraising the West: Other'
 'URW: Overpraising the West: The West belongs in the right side of history'
 'URW: Overpraising the West: The West has the strongest international support'
 'URW: Praise of Russia: Other'
 'URW: Praise of Russia: Praise of Russian President Vladimir Putin'
 'URW: Praise of Russia: Praise of Russian military might'
 'URW: Praise of Russia: Russia has international support from a number of countries and people'
 'URW: Praise of Russia: Russia is a guarantor of peace and prosperity'
 'URW: Praise of Russia: Russian invasion has strong national support'
 'URW: Russia is the Victim: Other'
 'URW: Russia is the Victim: Russia actions in Ukraine are only self-defence'
 'URW: Russia is the Victim: The West is russophobic'
 'URW: Russia is the Victim: UA is anti-RU extremists'
 'URW: Speculating war outcomes: Other'
 'URW: Speculating war outcomes: Russian army is collapsing'
 'URW: Speculating war outcomes: Ukrainian army is collapsing'

 The (CC) climate change narratives are:
 'CC: Amplifying Climate Fears: Amplifying existing fears of global warming'
 'CC: Amplifying Climate Fears: Doomsday scenarios for humans'
 'CC: Amplifying Climate Fears: Earth will be uninhabitable soon'
 'CC: Amplifying Climate Fears: Other'
 'CC: Amplifying Climate Fears: Whatever we do it is already too late'
 'CC: Climate change is beneficial: CO2 is beneficial'
 'CC: Controversy about green technologies: Other'
 'CC: Controversy about green technologies: Renewable energy is costly'
 'CC: Controversy about green technologies: Renewable energy is dangerous'
 'CC: Controversy about green technologies: Renewable energy is unreliable'
 'CC: Criticism of climate movement: Ad hominem attacks on key activists'
 'CC: Criticism of climate movement: Climate movement is alarmist'
 'CC: Criticism of climate movement: Climate movement is corrupt'
 'CC: Criticism of climate movement: Other'
 'CC: Criticism of climate policies: Climate policies are ineffective'
 'CC: Criticism of climate policies: Climate policies are only for profit'
 'CC: Criticism of climate policies: Climate policies have negative impact on the economy'
 'CC: Criticism of climate policies: Other'
 'CC: Criticism of institutions and authorities: Criticism of international entities'
 'CC: Criticism of institutions and authorities: Criticism of national governments'
 'CC: Criticism of institutions and authorities: Criticism of political organizations and figures'
 'CC: Criticism of institutions and authorities: Criticism of the EU'
 'CC: Criticism of institutions and authorities: Other'
 'CC: Downplaying climate change: CO2 concentrations are too small to have an impact'
 'CC: Downplaying climate change: Climate cycles are natural'
 'CC: Downplaying climate change: Human activities do not impact climate change'
 'CC: Downplaying climate change: Humans and nature will adapt to the changes'
 'CC: Downplaying climate change: Ice is not melting'
 'CC: Downplaying climate change: Other'
 'CC: Downplaying climate change: Temperature increase does not have significant impact'
 'CC: Downplaying climate change: Weather suggests the trend is global cooling'
 'CC: Green policies are geopolitical instruments: Green activities are a form of neo-colonialism'
 'CC: Green policies are geopolitical instruments: Other'
 'CC: Hidden plots by secret schemes of powerful groups: Blaming global elites'
 'CC: Hidden plots by secret schemes of powerful groups: Climate agenda has hidden motives'
 'CC: Hidden plots by secret schemes of powerful groups: Other'
 'CC: Questioning the measurements and science: Data shows no temperature increase'
 'CC: Questioning the measurements and science: Greenhouse effect/carbon dioxide do not drive climate change'
 'CC: Questioning the measurements and science: Methodologies/metrics used are unreliable/faulty'
 'CC: Questioning the measurements and science: Other'
 'CC: Questioning the measurements and science: Scientific community is unreliable'
"""

def GPT4o_classifier(news_article:str):
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format= GENERALMODEL,
        messages=
        [
            {"role":"system", "content": classifier_prompt},
            {"role":"user","content": news_article}
        ]
    )
    return response

In [39]:
article = """Ukraine's Fate Will Be Decided In Coming Year, Top Zelensky Aide Admits 

 Ukraine's Fate Will Be Decided In Coming Year, Top Zelensky Aide Admits

In surprisingly blunt words, a top aide to Ukrainian President Volodymyr Zelensky has warned that the coming year will essentially decide the fate of Ukraine and its war with Russia.

"A turning point in the war is approaching," Andrii Yermak, who serves as chief of staff for the Office of the President of Ukraine, said Monday. "The next year will be decisive in this regard." He issued the words while appealing for more urgent aid from Washington in an address to the hawkish DC-based Hudson Institute think tank.

Yermak sought to assure the audience that Zelensky has "a clear plan" forward even as Western media has by and large soured on Kiev's prospects for success. Much of this is about Zelensky sending envoys to do damage control in Washington at a moment the US administration's focus is off Ukraine and on Gaza events instead.

He described advancing plans for "the development of our defense industry, and the deploying of our own arms production. But [that] will be later."

But he quickly pivoted to an immediate need for more "weapons right now"--describing that "Russia still has air superiority. It is still capable of producing missiles, doing evasion of sanctions…And we especially need air defense systems."

Without doubt, the Zelensky admin is in damage control after eyebrow-raising comments were issued to The Economist early this month by Ukraine's top commander, who admitted there will be no breakthrough and the battlefield situation is in a stalemate. The New York Times had characterized his remarks as "the first time a top Ukrainian commander said the fighting had reached an impasse."

So now Zelensky appears to be dispatching his envoys to calm Washington jitters over all the "bad news" of late out of Ukraine.

Yermak also sought to assure the Hudson Institute conference that more billions given to Ukraine won't be "charity" but is instead an "investment" in America's "global leadership."

He further emphasized Zelensky's continued rejection of ceasefire talks with Russia, unless it's purely on Kiev's terms. "We seek peace, but not just any peace. In our case, ending the war through compromise is nothing more than pausing it. Ukraine will not repeat the mistake of Minsk," Yermak said.

Watch the full Hudson Institute speech below:"""

classification = GPT4o_classifier(article)

#classes: URW: Speculating war outcomes: Other;URW: Discrediting Ukraine: Situation in Ukraine is hopeless;URW: Discrediting the West, Diplomacy: West is tired of Ukraine;URW: Praise of Russia: Praise of Russian military might;URW: Discrediting the West, Diplomacy: The West does not care about Ukraine, only about its interests

for classi in classification.choices[0].message.parsed.narratives:
    print(classi)

categories.DISCREDIT_UA_HOPELESS
categories.WAR_OUTCOMES_UA_COLLAPSING


In [40]:
print(classification.choices)

[ParsedChoice[GENERALMODEL](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[GENERALMODEL](content='{"narratives":["URW: Discrediting Ukraine: Situation in Ukraine is hopeless","URW: Speculating war outcomes: Ukrainian army is collapsing"],"explanation":"The narrative highlights difficulties faced by Ukraine, depicting the situation as dire and possibly hopeless, especially with no breakthrough in combat. It suggests concerns about Ukraine\'s ability to win, linking it to a broader issue of Western loss of confidence, supported by Yermak and Zelensky\'s urgent pleas for aid. The article underscores the perception of a military stalemate and potential collapse without immediate support, aligning with both cited sub-narratives."}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=GENERALMODEL(narratives=[<categories.DISCREDIT_UA_HOPELESS: 'URW: Discrediting Ukraine: Situation in Ukraine is hopeless'>, <categories.WAR_O

Okay so the model predicts stuff, let's make a pipeline to have it predict the train set. Since the openai models cannot learn (at least not this version), so lets make some predictions for the entire train set. First, let's import the dataset and the mlb models.

In [43]:
df = pd.read_csv("data.csv")

# load the mlb files back in, to get the classes and transform functions
with open("dom_mlb.pkl", "rb") as f:
    dom_mlb = pickle.load(f)

with open("sub_mlb.pkl", "rb") as f:
    sub_mlb = pickle.load(f)

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Batch API Calls are cheaper, so lets format the data in a way where we can pass them as a batch to the openai api.